In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import natsort
import glob
import numpy as np
import pandas as pd
import torchvision.models.vgg as vgg
import itertools
from PIL import Image

import torch.optim as optim
from torch.utils.data import Dataset,DataLoader

import torchvision
import torchvision.transforms as transforms

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1218)
if device =='cuda':
    torch.cuda.manual_seed_all(1218)

In [ ]:
resnet18_pretrained = models.resnet18(pretrained=True)

In [ ]:
num_class = 3
num_ftrs = resnet18_pretrained.fc.in_features
resnet18_pretrained.fc = nn.Linear(num_ftrs, num_class)
resnet18_pretrained.to(device)

In [ ]:
trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224)),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
train_data = torchvision.datasets.ImageFolder('your_path',transform=trans)

In [ ]:
data_loader = DataLoader(dataset = train_data, batch_size = 32, shuffle = True)

In [ ]:
resnet18_pretrained
a=torch.Tensor(1,3,224,224).to(device)
out = resnet18_pretrained(a)
print(out)

In [ ]:
net=Net().to(device)

In [ ]:
optimizer = optim.Adam(resnet18_pretrained.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss().to(device)
lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.005)

In [ ]:
epochs = 10

for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    lr_sche.step()
    for i, data in enumerate(data_loader, 0):
        # get the inputs
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = resnet18_pretrained(inputs)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 30 == 29:    # print every 30 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 30))
            running_loss = 0.0
    
    
    

print('Finished Training')

In [ ]:
torch.save(resnet18_pretrained.state_dict(), '/kaggle/output.pth')

In [ ]:
resnet18_pretrained = resnet18_pretrained.to(device)

In [ ]:
resnet18_pretrained.load_state_dict(torch.load('/kaggle/output.pth'))

In [ ]:
class CustomDataSet(Dataset):

    def __init__(self, main_dir, transform):
        self.main_dir = main_dir
        self.transform = transform

        all_imgs = os.listdir(main_dir)
        self.total_imgs = natsort.natsorted(all_imgs)

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])

        image = Image.open(img_loc).convert("RGB")

        tensor_image = self.transform(image)

        return tensor_image


In [ ]:
test_data = CustomDataSet('your_path', transform=trans)

In [ ]:
test_set = DataLoader(dataset = test_data, batch_size = 32)

In [ ]:
result =[]
with torch.no_grad():
    file=pd.read_csv('your_path.csv')
    
    for data in test_set:
        imgs = data
        imgs = imgs.to(device)
        prediction = resnet18_pretrained(imgs)
        result.append(torch.argmax(prediction,1).tolist())
        

    result_ = list(itertools.chain.from_iterable(result))
    prediction_np = np.array(result_)
    print(result_)

    file['predictions'] =result_
    file.to_csv('result.csv', index=False)       